# 0. Librairies utilisées

In [1]:
# pip install dython redémarrer jupyter après
from dython.nominal import associations, identify_nominal_columns

from imblearn.metrics import classification_report_imbalanced
# Oversampling
from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from scipy.stats import pearsonr, chi2_contingency
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import statsmodels.api
import warnings


%matplotlib inline

# Sommaire

- ### [1. Exploration des données](#1)
- ### [2. Nettoyage des données](#2)
- ### [3. Analyse descriptive](#3)
- ### [3.1. Liaisons entre les variables](#3.1)
- ### [3.2. Analyse des liaisons entre les variables catégorielles](#3.2)
- ### [3.3. Liaison entre variables quantitatives et qualitatives](#3.3)
- ### [4. Implémentation des algorithmes de ML](#4)


<a class="anchor" name="1"></a>
# 1. Exploration des données

In [2]:
df = pd.read_csv("strokes.csv", index_col="id")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'strokes.csv'

In [ ]:
print("- Nombre d'observations et de variables : ", df.shape,'\n')
print("- Variables explicatives/expliquée et type : ")
df.info()

<div class= "alert alert-info">
    - C'est un jeu de données avec un nombre d'observations assez bas : 5110
</div>

In [ ]:
# Variable Age

print("- Statistiques sur la variable 'age' :\n", df['age'].describe(),'\n')

df.sort_values(by = 'age', ascending = True).head(500)
print("Ages entre 0 et 2 ans :\n", df[(df.age%1 != 0.00) & (df.age<2)][['age', 'stroke']])
# Entre 0 et 2 ans, les âges sont exprimés avec des nombres décimaux, voir par la suite si c'est à corriger ou pas
# pour l'implémentation des algorithmes de ML. 

# Classes d'âge
df_ca = df.copy()
df_ca['age_classes'] = pd.qcut(df['age'], labels=[0, 1, 2, 3], q=4)

# répartition des attaques selon la classe d'âge
stroke_groupby = df_ca.groupby(['age_classes', 'stroke']).agg({'stroke' : 'count'})
stroke_groupby


<div class= "alert alert-info">
    - L'écart type, les quantiles et la moyenne nous montrent qu'il y a une répartition des âges homogène entre 0 et 82 ans.</br>
    - Il y a de plus en plus de cas de crises cardiaques dans les classes d'âge des personnes les plus âgées, c'est logique.
</div>

In [ ]:
print("Gender :\n", df['gender'].value_counts(normalize=True),'\n')
print("hypertension :\n", df['hypertension'].value_counts(normalize=True),'\n')
print("work_type :\n", df['work_type'].value_counts(normalize=True),'\n')

<div class= "alert alert-info">
     Dans le jeu de données : </br>
    - La proportion de femmes est légèrement plus importante.</br>
    - Très peu de cas d'hypertension.</br>
    - Une majorité de jobs privés.</br>   
</div>

In [ ]:
#Stroke analyse

print(df['stroke'].isna().sum())
print("stroke :\n", df['stroke'].value_counts(normalize=True),'\n')
#aucune valeur vide

df['stroke'].value_counts()
#249 crise cardiaque décelé
#4861 sans crise cardiaque


<div class= "alert alert-info">
    - Très faible proportion d'observations avec la variable cible 'stroke' positive donc peu de cas de crise cardiaque, attention au déséquilibre des classes pour la variable cible. Comme le nombre d'observations est faible, nécessité de faire de <b>l'oversampling</b>.   
</div>

In [ ]:
#Type de résidence
print(df['Residence_type'].isna().sum())
print("Residence_type :\n", df['Residence_type'].value_counts(normalize=True),'\n')
#aucune valeur vide

df['Residence_type'].value_counts()

print (df['Residence_type'].value_counts())

#Urban    2596
#Rural    2514

In [ ]:
#Taux de glucose
print(df['avg_glucose_level'].isna().sum())
#aucune valeur vide

df['avg_glucose_level'].describe()

#taux moyen de glucose 106.147677

<a class="anchor" name="2"></a>
# 2. Nettoyage des données

In [ ]:
df.index.duplicated().sum()

<div class= "alert alert-info">
    Il n'y a pas de doublons au niveau de l'index : pas besoin de supprimer de doublons
</div>

In [ ]:
df.isna().sum()

<div class= "alert alert-info">
    Il y a des valeurs NaN dans la colonne bmi que l'on va remplacer par un mean
</div>

In [ ]:
#On remplace les valeurs NA par le mean de la colonne bmi
df_new = df.copy()

df_new.loc[:, ("bmi")] = df["bmi"].fillna(df["bmi"].mean())

#On remplace également les valeurs Unknown par le mode de la colonne
df_new = df_new.replace("Unknown", np.NaN)
df_new["smoking_status"] = df_new["smoking_status"].fillna(df_new["smoking_status"].mode()[0])
df_new.head()

In [ ]:
df_new["bmi"].describe()

<a class="anchor" name="3"></a>
# 3. Analyse descriptive

In [ ]:
#Est-ce que le fait d'être marié peut avoir un impact sur la crise cardiaque ?
pd.crosstab(df_new["ever_married"], df_new["stroke"])

In [ ]:
#La même répartition en %
round(pd.crosstab(df_new["ever_married"], df_new["stroke"], normalize="index")*100,2)

In [ ]:
#Est-ce que le fait d'être fumeur peut avoir un impact sur la crise cardiaque ?
round(pd.crosstab(df_new["smoking_status"], df_new["stroke"],normalize="index")*100, 2)

#On constate un écart non significatif entre les fumeurs et les non fumeurs sur le risque de crise cardiaque.

In [ ]:
#Est-ce que le bmi peut avoir un impact sur la crise cardiaque ?

df_bm1c = df_new.copy()
df_bm1c["bmi_classes"] = pd.qcut(df_new["bmi"], labels=[0,1,2,3], q=4)
round(pd.crosstab(df_bm1c["bmi_classes"], df_bm1c["stroke"],normalize="index")*100, 2)

In [ ]:
df_new["bmi"]

In [ ]:
#Est-ce que le type de travail peut avoir un impact sur la crise cardiaque ?
round(pd.crosstab(df_new["work_type"], df_new["stroke"],normalize="index")*100, 2)


<div class= "alert alert-info">
  Les cas de crise cardiaque les plus fréquents se trouvent chez les salariés du privé.</br>
  Aucun cas chez ceux qui n'ont jamais travaillé.
</div>

In [ ]:
#Est-ce que le genre peut avoir un impact sur la crise cardiaque ?
round(pd.crosstab(df_new["gender"], df_new["stroke"],normalize="index")*100, 2)


<div class= "alert alert-info">
  Un peu plus de crise cardiaque chez les hommes, tout en sachant qu'il y a un nombre plus important d'observation chez les femmes (58,5 %).
</div>

In [ ]:
 # Est-ce que le type de résidence est un vecteur marquant sur la crise cardiaque ?
colonne1 = df_new['Residence_type']
colonne2 = df_new['stroke']


print("")

pd.crosstab(colonne1, colonne2)


<div class= "alert alert-info">
    Un petit peu plus de crise cardiaque en ville 135 contre 114 en campagne
 </div>

In [ ]:
#Taux de glucose en fonction de l'habitation

# Quantité maximale
max_avg_glucose_level = lambda avg_glucose_level: avg_glucose_level[avg_glucose_level > 0].max()

# Quantité minimale
min_avg_glucose_level = lambda avg_glucose_level: avg_glucose_level[avg_glucose_level > 0].min()

# Quantité médiane
median_avg_glucose_level = lambda avg_glucose_level : avg_glucose_level[avg_glucose_level > 0].median()




# Définition du dictionnaire de fonctions à appliquer
functions_to_apply = {
    'avg_glucose_level' : [max_avg_glucose_level, min_avg_glucose_level, median_avg_glucose_level]
}


# Operation groupby
avg_glucose_level_groupby = df_new.groupby('Residence_type').agg(functions_to_apply)

# Je renomme les colonnes produite par le groupby
avg_glucose_level_groupby.columns.set_levels(['max_avg_glucose_level', 'min_avg_glucose_level', 'median_avg_glucose_level'], level=1, inplace = True)



# Affichage des premières lignes du Dataframe produit par l'opération groupby
avg_glucose_level_groupby.head()



<div class= "alert alert-block alert-info">
les taux de glucose sont presque équivalent en ville et en campagne
</div>

In [ ]:
df_avg_gluc_c = df_new.copy()
df_avg_gluc_c['avg_glucose_level_classes'] = pd.cut(df_new['avg_glucose_level'], bins=[0,70,100,125,np.inf],labels=['Hypoglycémie','Taux normal','Hyperglycémie modérée','Diablétique'])

df_avg_gluc_c['avg_glucose_level_classes']


In [ ]:
df_age_c = df_new.copy()
df_age_c =df_age_c[df_age_c['stroke']==1]
df_age_c['age_c'] = pd.cut(df_new['age'], bins=[0,14,24,64,np.inf],labels=['Enfants','Adolescents ','Adultes','Aînés'])

df_age_c['age_c']

In [ ]:
state_summary_age=df_age_c.groupby(['age_c','gender']).agg( {"stroke": 'count'})

state_summary_age

<div class= "alert alert-block alert-info">
Hypoglycémie : Inférieur à 70g/dL de sang </br>
Le taux normal de la glycémie à jeun oscille entre 70 mg/dL et 100 g/dL </br>
Hyperglycémie modérée : Entre 1 et 1.25g/L </br>
Au dessus de 126mg à jeune la personne est considéré comme diablétique </br></br>
Les données sont donc divisées en 4 quantile 0:70  < 70:100 < 100:125 < 125: </br>
</div>

In [ ]:
state_summary=df_avg_gluc_c.groupby(['avg_glucose_level_classes','gender']).agg( {"gender": 'count'})
state_summary

#nb_gender=df.groupby(['gender']).agg( {"gender": 'count'})
#nb_gender
#Etude des crises cardiaques qui dépend de sa tranche de son taux de glucose

#60% Femme 40%Homme

<div class= "alert alert-block alert-info">
Répartition équilibré des taux de glucose sachant qu'il y 800 femmes de plus </br>

</div>

In [ ]:
df_avg_gluc_c = df_new.copy()
df_avg_gluc_c =df_avg_gluc_c[df_avg_gluc_c['stroke']==1]
df_avg_gluc_c['avg_glucose_level_classes'] = pd.cut(df_new['avg_glucose_level'], bins=[0,70,100,125,np.inf],labels=['Hypoglycémie','Taux normal','Hyperglycémie modérée','Diablétique'])

df_avg_gluc_c


<div class= "alert alert-block alert-info">
Le type d'habitation n'influence pas le taux de glucose dans le sang </br>

</div>

In [ ]:



state_summary=df_avg_gluc_c.groupby(['avg_glucose_level_classes','g_Male']).agg( {"stroke": 'count'})
state_summary




<div class= "alert alert-block alert-info">
Moins de crise cardiaque sur les cas en Hypoglycemie et sur ceux en Hyperglycémie modérée</br>

</div>



In [ ]:
# Analyse de la variable avg_glucose_level

#Séparation de la variable
avg_glucose_level_studies=df_new[["avg_glucose_level"]]

#Calcul de la moyenne de la variable
stats=pd.DataFrame(avg_glucose_level_studies.mean(), columns=['moyenne'])
stats.round(2)

#Calcul de la median de la variable
stats['median']=avg_glucose_level_studies.median()

#Calcul de la différence entre la moyenne et la median de la variable
stats['mean_med_diff'] = abs(stats['moyenne'] - stats['median'])
stats.round(2)

#Calcul du quantile de la variable
quantile=avg_glucose_level_studies["avg_glucose_level"].quantile(q = [0.25,0.5,0.75])
quantile

stats[['q1', 'q2', 'q3']] = avg_glucose_level_studies.quantile(q=[0.25, 0.5, 0.75]).transpose()

#Calcul du max et minimum et de la différence de la variable
stats['min'] = avg_glucose_level_studies.min()
stats['max'] = avg_glucose_level_studies.max()
stats['min_max_diff'] = stats['max'] - stats['min']

stats

<a class="anchor" name="3.1"></a>
## 3.1 Liaisons entre les variables

In [ ]:
df_new.head()

## 3.1 Analyse des liaisons entre les variables continues

In [ ]:



dfnum=df_new[['age','avg_glucose_level','bmi']]
dfnum.corr()




<div class= "alert alert-block alert-info">
    Compte-tenu des coefficients de corrélation plus proche de 0 que de 1, les variables semblent peu corrélées entre elles. 
</div>

In [ ]:

# la fonction corr() permet de récupérer les coefficients de corélation entre les variables,
# La p-value permet de dire si les variables sont indépendantes alors que le coef de corrélation permet de dire à quel point
# elles sont corrélées positivement ou négativement.

print(pd.DataFrame(pearsonr(df_new['age'], df_new['avg_glucose_level']), index=['pearson_coeff','p-value'], columns=['age/avg_glucose_level']))
print(pd.DataFrame(pearsonr(df_new['age'], df_new['bmi']), index=['pearson_coeff','p-value'], columns=['age/bmi']))
print(pd.DataFrame(pearsonr(df_new['avg_glucose_level'], df_new['bmi']), index=['pearson_coeff','p-value'], columns=['avg_glucose_level/bmi']))

In [ ]:
corr = df_new.corr()
corr.style.background_gradient(cmap="coolwarm").format(precision=2)


<div class= "alert alert-block alert-info">

La variable qui a la corrélation la plus forte avec la crise cardiaque est l'âge, celle qui a le moins de corrélation contrairement à ce que l'on pourrait supposer est le bmi

</div>

In [ ]:
df_plot = df_new[["stroke","bmi"]]

In [ ]:
df_plot["q"] = pd.qcut(df_plot["bmi"], q=4).cat.codes

In [ ]:
df_plot = df_plot.drop(columns="bmi")

In [ ]:
df_plot = df_plot.groupby(["q", "stroke"]).size().reset_index(name='counts')

In [ ]:
#Découper les BMI par groupe de "obésité" vs "sain"

labels = df_plot["q"].unique().tolist()
labels = ["q"+str(i) for i in labels]
#labels = 'q'.join(str(labels) for n in labels)
stroke = df_plot["counts"].loc[df_plot["stroke"]==1].tolist()
no_stroke = df_plot["counts"].loc[df_plot["stroke"]==0].tolist()

width = 0.35

fig, ax = plt.subplots()

ax.bar(labels, no_stroke, width, label = "no_Stroke")
ax.bar(labels, stroke, width, bottom = no_stroke, label = "Stroke")

ax.set_ylabel("nb")
ax.set_xlabel("Groupe BMI")
ax.legend()

plt.show()

<div class= "alert alert-block alert-info">
Le fait d'être dans une tranche de bmi élevée n'augmente que de peu le risque d'avoir une crise cardiaque.
</div>

<a class="anchor" name="3.2"></a>
## 3.2 Analyse des liaisons entre les variables catégorielles

In [ ]:
dfcat=df_new[['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status','stroke']]


table = pd.crosstab(dfcat['work_type'],dfcat['smoking_status'])

resultats_test = chi2_contingency(table)
statistique = resultats_test[0]
p_valeur = resultats_test[1]
degre_liberte = resultats_test[2]
la_liste = resultats_test[3]
print("- Statistique : ", statistique, "\n- P-value : ", p_valeur, "\n- Degré de liberté : ",degre_liberte)

table

<div class= "alert alert-block alert-info">
    P-Value < 5% donc on rejette H0, les variables work_type et Smoking_status sont dépendantes 
</div>

In [ ]:
def V_cramer(cont_table, N):
    k = cont_table.shape[0]
    r = cont_table.shape[1]
    k_tilde = k - (k-1)**2/(N-1)
    r_tilde = r - (r-1)**2/(N-1)
    num = max(0, statistique/N-((k-1)*(r-1))/(N-1))
    denom = min(k_tilde-1, r_tilde-1)
    v_cramer=np.sqrt(num/denom)
    return v_cramer

print("- V_Cramer : ", V_cramer(table, df_new.shape[0]))


<div class= "alert alert-block alert-info">
    Le V_Cramer n'est pas très élevé (plus proche de 0 que de 1), les variables work_type et Smoking_status ne sont pas très corrélées sans que ce soit négligeable.
</div>

In [ ]:
# Ajout Nathalie : Liaison entre heart_disease et stroke
table = pd.crosstab(dfcat['heart_disease'],dfcat['stroke'])
resultats_test = chi2_contingency(table)
statistique = resultats_test[0]
p_valeur = resultats_test[1]
degre_liberte = resultats_test[2]
la_liste = resultats_test[3]
print("- Statistique : ", statistique, "\n- P-value : ", p_valeur, "\n- Degré de liberté : ",degre_liberte)

print("- V_Cramer : ", V_cramer(table, df.shape[0]))

<div class= "alert alert-block alert-info">
    P-Value < 5% donc on rejette H0, les variables heart_disease et stroke sont dépendantes.</br>
    Le V_Cramer n'est pas très élevé (plus proche de 0 que de 1), les variables heart_disease et stroke ne sont pas très corrélées sans que ce soit négligeable.
</div>

In [ ]:
#https://medium.com/@knoldus/how-to-find-correlation-value-of-categorical-variables-23de7e7a9e26

categorical_features=identify_nominal_columns(df)
categorical_features
['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status','stroke']



complete_correlation= associations(df_new, filename= 'complete_correlation.png', figsize=(10,10))


df_complete_corr=complete_correlation['corr']
df_complete_corr.dropna(axis=1, how='all').dropna(axis=0, how='all').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)



categorical_correlation= associations(dfcat, filename= 'categorical_correlation.png', figsize=(10,10))

<a class="anchor" name="3.3"></a>
## 3.3 Liaison entre variables quantitatives et qualitatives

In [ ]:
# Analyse de l'influence de age sur stroke
result = statsmodels.formula.api.ols('stroke ~ age', data=df).fit()
table = statsmodels.api.stats.anova_lm(result)

table

<div class= "alert alert-block alert-info">
    La p-value (PR(>F)) est inférieur à 5% donc on rejette l'hypothèse selon laquelle l'âge n'influe pas sur stroke.
</div>

## Festival de graphique

In [ ]:
N1 = len(df_new[df_new['stroke']==0])
N2 = len(df_new[df_new['stroke'] == 1])


plt.bar([1],[N1], color = ['green'], width =  [0.6,0.5])
plt.bar([2],[N2], color = ['red'], width =  [0.6,0.5])

plt.xticks([1,2], ['Sans Crises Cardiaque','Crises Cardiaque'])


plt.ylabel('Nombre de cas')


plt.text(1,N1,N1)
plt.text(2,N2,N2)
plt.legend();

In [ ]:
#N1 = len(state_summary_age[state_summary_age['age_c']=='Enfants'])
state_summary_age_c=df_age_c.groupby(['age_c']).agg( {"stroke": 'count'})
state_summary_age_c



N1 = len(df_avg_gluc_c[df_avg_gluc_c['avg_glucose_level_classes']=='Hypoglycémie'])
N2 = len(df_avg_gluc_c[df_avg_gluc_c['avg_glucose_level_classes']=='Taux normal'])
N3 = len(df_avg_gluc_c[df_avg_gluc_c['avg_glucose_level_classes']=='Hyperglycémie modérée'])
N4 = len(df_avg_gluc_c[df_avg_gluc_c['avg_glucose_level_classes']=='Diablétique'])


plt.bar([1],[N1], color = ['green'], width =  [5,1])
plt.bar([15],[N2], color = ['blue'], width =  [5,1])
plt.bar([30],[N3], color = ['orange'], width =  [5,1])
plt.bar([45],[N4], color = ['red'], width =  [5,0.2])

plt.xticks([1,15,30,45], ['Hypoglycémie','Taux normal','Hyperglycémie modérée','Diablétique'])


plt.ylabel('Nombre de cas')


In [ ]:
N1 = len(df_new[df_new['Residence_type']=='Rural'])
N2 = len(df_new[df_new['Residence_type'] == 'Urban'])


plt.bar([1],[N1], color = ['green'], width =  [0.6,0.5])
plt.bar([2],[N2], color = ['grey'], width =  [0.6,0.5])

plt.xticks([1,2], ['Rural','Urban'])


plt.ylabel('Nombre de cas')


plt.text(1,N1,N1)
plt.text(2,N2,N2)
plt.legend();

In [ ]:
df_ca_pos = df_ca[df_ca.stroke==1].groupby(['age_classes']).agg({'stroke' : 'count'})
df_ca_pos


In [ ]:
# Représentation du nombre de cas positifs pour chaque classe d'âge
#df_ca_pos.plot(kind='bar', title='Représentation du nombre de cas positifs pour chaque classe d âge', legend=True)
#axes.set_xlabel('axe des x');
plt.figure(figsize=(8,8))
plt.bar(range(4), df_ca_pos.stroke , color = 'green', width = 0.6)
plt.xlabel('Classes d âge')
plt.ylabel('Nombre de crises cardiaques')
plt.xticks(range(4), ["De 0 à 25 ans","De 26 à 45 ans","De 46 à 61 ans","De 62 à 82 ans"])
plt.title('Représentation du nombre de cas positifs pour chaque classe d âge');

<a class="anchor" name="4"></a>
# 4.Implémentation des algorithmes de Machine Learning

## a. Régression Logistique

In [ ]:
# Numérisation des variables catégorielle pour les utiliser dans un algo de ML

# binarisation des variables catégorielles non hiérarchisées avec plus de 2 valeurs possibles
df_rl = pd.get_dummies(df_new, prefix=['g', 'wt', 'ss'], columns=['gender', 'work_type', 'smoking_status'], drop_first=True)
df_rl

In [ ]:
# Numérisation des variables catégorielles binaires

df_rl.ever_married.replace(['Yes','No'], [0, 1], inplace=True)
df_rl.Residence_type.replace(['Rural','Urban'], [0, 1], inplace=True)

# Séparation des données en variables explicatives et variable cible
X = df_rl.drop("stroke", axis=1)
y = df_rl.stroke

In [ ]:
# Standardisation des données
# On instancie StandardScaler
scaler=StandardScaler()
X_scaled_=scaler.fit(X).transform(X)
X_scaled=pd.DataFrame(X_scaled_)

In [ ]:
# Séparation des données en jeu de d'entrainement et de test
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.3,random_state=42)
df_rl.head()

In [ ]:
# sans oversampling
# Entraînement du modèle de régression logistique

lr = LogisticRegression()
lr.fit(X_train, y_train)

# Affichage des résultats
y_pred = lr.predict(X_test)

# Coefficients obtenus
coeff=lr.coef_

# On crée un dataframe qui combine à la fois variables et coefficients 
resultats=pd.DataFrame(X.columns, columns=["Variables"])
resultats['Coefficients']=coeff[0].tolist()

resultats['Odd_Ratios']=np.exp(coeff).tolist()[0]
resultats.sort_values(by = 'Odd_Ratios',ascending = False).head(15)

<div class= "alert alert-block alert-info">
L'âge a l'Odd_Ratios le plus élevé : lorsque l'âge augmente de 1, cela augmente d'environ 5,5 fois les chances d'avoir une crise cardiaque.
Arrivent ensuite bien loin derrière le niveau de glucose et l'hypertension.
Toutes les variables proches de 0 n'ont que très peu d'influence.
Pour les variables catégorielles, celle qui semble la plus impactante est le working_status.
</div>

In [ ]:

# Evaluation du modèle avec les métriques
print("Accuracy : ", accuracy_score(y_test, y_pred))
print("Balanced Accuracy : ", balanced_accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

<div class= "alert alert-block alert-info">
L'accuracy est excellente mais la Balanced accuracy est moins bonne, le déséquilibre des classes influence donc les performances du modèle pour trouver les cas positifs, il est nécessaire de faire de l'oversampling.
</div>

In [ ]:
# Sur-échantillonnage
rOs = RandomOverSampler(sampling_strategy='minority')
X_ro, y_ro = rOs.fit_resample(X_train, y_train)
X_ro.shape
y_ro.value_counts()

In [ ]:
# Régression Logistique
warnings.filterwarnings('ignore')

# Détermination des hyperparamètres du modèle les plus optimum 
parameters = {
    'penalty' : ['l1','l2'], # l1 lasso l2 ridge
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}

logreg=LogisticRegression()
lr1 = GridSearchCV(logreg, param_grid = parameters, scoring='accuracy',cv=7)

# Entraînement du modèle de régression logistique
lr1.fit(X_ro, y_ro)

print("tuned hyperparameters :(best parameters) ",lr1.best_params_)
print("accuracy :",lr1.best_score_)

warnings.filterwarnings('default')

In [ ]:
# Affichage des résultats
y_pred = lr1.predict(X_test)

# Evaluation du modèle avec les métriques
#print(classification_report_imbalanced(y_test, y_pred))
print(classification_report(y_test, y_pred))

print("Accuracy : ", accuracy_score(y_test, y_pred))
print("Balanced Accuracy : ", balanced_accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

<div class= "alert alert-block alert-info">
L'accuracy est moins bonne que dans le premier test mais la Balanced accuracy est meilleure, on obtient une qualité de prévision plus équilibrée entre les 2 classes.
</div>

### b. SVC

In [ ]:
from sklearn.pipeline import Pipeline
SVCpipe = Pipeline([('scale', StandardScaler()),
                   ('SVC',LinearSVC())])

# Gridsearch to determine the value of C
param_grid = {'SVC__C':np.arange(0.01,100,10)}
linearSVC = GridSearchCV(SVCpipe,param_grid,cv=5,return_train_score=True)


linearSVC.fit(X_ro, y_ro)
print(linearSVC.best_params_)

score = linearSVC.score(X_ro, y_ro)
print("Score: ", score)

#cv_scores = cross_val_score(linearSVC, X_ro, y_ro, cv=10)
#print("CV average score: %.2f" % cv_scores.mean())




In [ ]:
ypred = linearSVC.predict(X_ro)

cm = confusion_matrix(y_ro, ypred)
print(cm)

In [ ]:
cr = classification_report(y_ro, ypred)
print(cr)

### c. PCA

In [ ]:
X

In [ ]:
X = df_new.drop(columns=["stroke","gender","ever_married","work_type","Residence_type","smoking_status"])
y = df_new["stroke"]

scaler = StandardScaler()
X_scaled_ = scaler.fit(X).transform(X)
X_scaled = pd.DataFrame(scaler.fit(X).transform(X))


liste = []

for i in range(1,6):
    pca = PCA(n_components=i)
    pca.fit(scaler.fit(X).transform(X))
    liste.append(sum(pca.explained_variance_ratio_))

fig,ax = plt.subplots(dpi=150)

ax.yaxis.set_ticks_position("left")
ax.xaxis.set_ticks_position("bottom")

plt.plot(np.arange(1,6),liste)
plt.xticks(np.arange(1,6, step=1))

plt.xlabel("Nombre de dimensions")
plt.ylabel("Somme des variances expliquées")

plt.show();

<div class= "alert alert-block alert-info">
On choisit 4 dimensions (~90% des variances expliquées par 4 dimensions)
</div>

In [ ]:
pca = PCA(n_components=4)
X_scaled_pca = pca.fit_transform(X_scaled)

In [ ]:
print(" Nombre de dimensions:",pca.n_components_,
      "\n \n Pourcentage de variance expliquée par chaque dimension:\n \n Dimension 1:",
      pca.explained_variance_ratio_[0],
      "\n Dimension 2:" ,pca.explained_variance_ratio_[1],
     "\n Dimension 3:" ,pca.explained_variance_ratio_[2],
     "\n Dimension 4:" ,pca.explained_variance_ratio_[3])


# On affiche l'influence de chaque variable dans chaque dimension. 
print("\nInfluence des variables pour chaque dimension: \n \n  ",pd.DataFrame(pca.components_,index=["PC1","PC2","PC3","PC4"],columns=X.columns))

<div class= "alert alert-block alert-info">
1ère dimension : Ce sont l'âge l'hypertension, le niveau de glucose et l'IMC qui ont le + d'importance
2ème dimension : C'est principalement l'IMC qui a le plus d'importance
3ème dimension : L'hypertension est la variable ayant le plus d'importance
4ème dimension : Ici c'est le niveau de glucose moyen qui a le plus d'importance
</div>